In [1]:
import numpy as np
import pandas as pd
import re # import regex 

In [2]:
df = pd.read_csv('netflix_titles.csv')
movies = np.array([{"title": row['title'], "description": row['description']} for _,row in df.iterrows()])[500:1000]

In [3]:

def tokenize(text):
    """
    Tokenizes the given text by splitting it into lowercase words.

    Args:
        text (str): The input text to be tokenized.

    Returns:
        list: A list of lowercase words extracted from the text.
    """
    return [x for x in re.findall(r"[a-z]+", text.lower())]

In [4]:
# tokenize the movie description and add it to our movie dictionaries
for movie in movies:
    movie["tokens"] = tokenize(movie["description"])

In [6]:
def build_inverted_index(movies):
    """
    Builds an inverted index from a list of movies.

    Args:
        movies (list): A list of movie dictionaries, where each dictionary contains a 'tokens' key
                       representing the tokens in the movie.

    Returns:
        dict: The inverted index dictionary, where each key is a token and the corresponding value
              is a list of tuples. Each tuple contains the index of the movie in the 'movies' list
              and the count of the token in that movie.
    """

    dic = {}
    for idx, doc in enumerate(movies):
        token_set = set(doc['tokens'])
        for token in token_set:
            if token not in dic:
                dic[token] = [(idx, doc['tokens'].count(token))]
            else:
                dic[token].append((idx, doc['tokens'].count(token)))
    return dic


In [7]:
def compute_idf(inv_idx, n_docs):
    """
    Compute the inverse document frequency (IDF) for each word in the given inverted index.

    Parameters:
    inv_idx (dict): The inverted index containing words as keys and their appearances as values.
    n_docs (int): The total number of documents.

    Returns:
    dict: A dictionary containing words as keys and their corresponding IDF values.
    """

    res = {}

    for word, appearance in inv_idx.items():
        df = len(appearance)
        df_ratio = df / n_docs
        
        idf = np.log2(n_docs/ (1 + df))
        res[word] = idf
    return res
    

In [8]:
def compute_doc_norms(index, idf, n_docs):
    """
    Compute the document norms based on the given index, inverse document frequency (idf), and number of documents (n_docs).

    Parameters:
    index (dict): A dictionary representing the index of words in the documents.
    idf (dict): A dictionary representing the inverse document frequency of words.
    n_docs (int): The total number of documents.

    Returns:
    numpy.ndarray: An array containing the computed document norms.
    """

    res = np.zeros(n_docs)

    for word, appearance in index.items():
        if word in idf:
            for doc_id, tf in appearance:
                res[doc_id] += (tf * idf[word]) ** 2
    return np.sqrt(res)


In [9]:
def accumulate_dot_scores(query_word_counts, index, idf):
    """
    Calculates the dot product scores between a query and documents in an index.

    Args:
        query_word_counts (dict): A dictionary containing the word counts of the query.
        index (dict): A dictionary containing the index of documents and their word counts.
        idf (dict): A dictionary containing the inverse document frequency (idf) values for each word.

    Returns:
        dict: A dictionary containing the accumulated dot product scores for each document in the index.
    """
    dic = {}
    for word, count in query_word_counts.items():
        if word in index and word in idf:
            query_tf = count * idf[word]

            for id, doc_tf in index[word]:
                product = query_tf * (doc_tf * idf[word])

                if id in dic:
                    dic[id] += product
                else:
                    dic[id] = product
    return dic


In [10]:
def search(query, index, idf, doc_norms, score_func=accumulate_dot_scores, tokenizer=tokenize):
    """
    Search for documents based on a given query.

    Args:
        query (str): The query string to search for.
        index (dict): The index of documents.
        idf (dict): The inverse document frequency values.
        doc_norms (dict): The document norms.
        score_func (function, optional): The scoring function to calculate dot scores. Defaults to accumulate_dot_scores.
        tokenizer (function, optional): The tokenizer function to tokenize the query. Defaults to tokenize.

    Returns:
        list: A list of tuples containing the score and document ID for each matching document.
    """
    query = query.lower()
    query_token = tokenizer(query)

    q_counts = {}
    for q in query_token:
        q_counts[q] = q_counts.get(q, 0) + 1
    
    q_tf_idf = {word: count * idf.get(word, 0) for word, count in q_counts.items()}

    q_norm = np.sqrt(sum(value ** 2 for value in q_tf_idf.values()))

    dot_scores = score_func(q_counts, index, idf)

    res = []

    for id, numerator in dot_scores.items():
        denominator = q_norm * doc_norms[id]
        if denominator:
            score = numerator / denominator
            res.append((score, id))
    res.sort(key = lambda x: -x[0])

    return res

In [11]:
inv_idx = build_inverted_index(movies) # build our inverted index

idf = compute_idf(inv_idx, len(movies))

inv_idx = {key: val for key, val in inv_idx.items() if key in idf}

doc_norms = compute_doc_norms(inv_idx, idf, len(movies)) #normalize our documents

In [18]:
results = search("Today is Sunday and I am hungry and tired and sleepy ", inv_idx, idf, doc_norms)

In [19]:
ans = results[:10] # get the top 10 results

In [22]:
for score, id in ans:
    print(score, ":" , {'title': movies[id]['title'], 'description': movies[id]["description"]})

0.2588426360357845 : {'title': 'Bheeshma', 'description': 'Tired of being single, a smart-aleck meme creator has a chance to prove himself to the girl he loves when he is suddenly appointed CEO of her workplace.'}
0.21307125991963727 : {'title': 'Best Leftovers Ever!', 'description': 'Decadent pasta from day-old fries? Skillful cooks transform tired leftovers into tantalizing new dishes in this wildly creative culinary competition.'}
0.11789467510812163 : {'title': 'Badha', 'description': 'A young woman who is suspected of practicing black magic and witchcraft is demonized and forced to live apart from her family.'}
0.07997631214893137 : {'title': 'Badla', 'description': 'When a woman is accused of killing her lover, a renowned lawyer is hired – but the more they try to  untangle the truth, the more convoluted it becomes.'}
0.0647422053257477 : {'title': 'Animas', 'description': "A teen's eerie visions become increasingly frequent and terrifying after her longtime friend gets a girlfri